In [25]:
import re
import math
import warnings
import tqdm
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from sympy import symbols, Eq, solve
from matplotlib.pyplot import MultipleLocator
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

In [26]:
data1=pd.read_excel('/root/GDUT/表1-患者列表及临床信息.xlsx')
# data2=pd.read_excel('/root/GDUT/表2-患者影像信息血肿及水肿的体积及位置.xlsx')
# data3=pd.read_excel('/root/GDUT/表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx')
# data4=pd.read_excel('/root/GDUT/表4-答案文件.xlsx')
data5=pd.read_excel('2b聚类数据.xlsx')
data6=pd.read_excel('2a数据_已替换时间戳_已矫正.xlsx')
# data_z=pd.read_excel("治疗方案.xlsx")

# data1.rename(columns={data1.columns[0]: "ID"}, inplace=True)
# data2.rename(columns={data2.columns[0]: "ID"}, inplace=True)

# data_f_1=pd.read_excel('附表1-检索表格-流水号vs时间.xlsx')
# data_f_time=pd.read_excel('时间点.xlsx')

In [27]:
data1[list(data1.columns)[16:]].shape

(160, 7)

In [28]:

data7 = data5

# 转换独热编码
xueya_list = []
for i in data7['血压']:
    y1 = i.split('/')[0]
    y2 = i.split('/')[1]
    if int(y1) > 139 or int(y1) < 90:
        status_i = 0
    elif int(y2) > 89 or int(y2) < 60:
        status_i = 0
    else:
        status_i = 1
    xueya_list.append(status_i)
data7['血压'] = np.array(xueya_list)
data7['性别_男'] = data5['性别'].apply(lambda x: 1 if x == '男' else 0)
data7['性别_女'] = data5['性别'].apply(lambda x: 1 if x == '女' else 0)
x = data7['年龄'].values
scaler = MinMaxScaler()
x_normalized = scaler.fit_transform(x.reshape(-1, 1))
binarizer = Binarizer(threshold=0.5)
x_binarized = binarizer.fit_transform(x_normalized)
data7['年龄'] = x_binarized

In [29]:
pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns',None)

#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

a=list(data7['脑出血前mRS评分'])
data7['脑出血前mRS评分']=[(i-min(a))/(max(a)-min(a)) for i in a]
data7.drop(columns={'性别'},axis=1,inplace=True)

data7 = pd.concat([data7, data1[list(data1.columns)[16:]]], axis=1)
data7.to_excel('2b数据处理结果.xlsx',index=False)
data7.shape

(160, 20)

In [30]:
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch, SpectralClustering
#  DBSCAN,
from sklearn.metrics import silhouette_score, calinski_harabasz_score


# 示例数据
data7=pd.read_excel('2b数据处理结果.xlsx')

data = data7[list(data7.columns)[13:]][:101]



In [31]:

# kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, random_state=0)
# 定义所需的聚类算法和参数
for c in [3,4,5]:
    print('*'*50,c,'*'*50)
    clustering_algorithms = [
        ("K-Means", KMeans(n_clusters=c)),
        ("Hierarchical Clustering", AgglomerativeClustering(n_clusters=c)),
        ("Birch", Birch(n_clusters=c)),
        ("Spectral Clustering", SpectralClustering(n_clusters=c))
        ]


    # 迭代聚类算法并分别计算评估指标
    for algorithm_name, algorithm in clustering_algorithms:
        algorithm.fit(data)
        labels = algorithm.labels_

        silhouette = silhouette_score(data, labels)
        ch_index = calinski_harabasz_score(data, labels)

        print("Clustering Algorithm:", algorithm_name)
        print("Clustering Labels:", labels)
        print("Silhouette coefficient:", silhouette)
        print("Calinski-Harabasz index:", ch_index)
        print("---------------------------------------")

************************************************** 3 **************************************************
Clustering Algorithm: K-Means
Clustering Labels: [0 2 0 2 2 1 2 2 2 0 1 2 1 2 2 0 0 2 2 2 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0]
Silhouette coefficient: 0.5774136514261422
Calinski-Harabasz index: 47.28457989006173
---------------------------------------
Clustering Algorithm: Hierarchical Clustering
Clustering Labels: [2 0 2 0 0 1 0 0 0 1 1 0 1 0 0 2 1 0 0 0 2 0 0 1 2 2 2 2 2 0 2 2 2 2 0 2 2
 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 1 2 0 1 1 1 2 2
 1 2 2 1 1 2 0 2 2 2 2 1 2 1 1 2 0 1 1 2 2 1 2 2 2 2 2]
Silhouette coefficient: 0.5970740088305504
Calinski-Harabasz index: 47.27722064040386
---------------------------------------
Clustering Algorithm: Birch
Clustering Labels: [1 1 1 2 2 0 1 2 1 0 0 1 0 2 1 1 0 1 2 1 1 1 2 0 1 1 1 1 1 1 1 1 1 1 1 

In [32]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# 示例数据
data7 = pd.read_excel('2b数据处理结果.xlsx')
data = data7[list(data7.columns)[3:10]]

# 定义所需的聚类算法和参数
clustering_algorithms = [
    ("K-Means", KMeans())  # 不需要指定初始参数
]

# 定义KMeans的超参数搜索范围
param_grid = {
    'n_clusters': [None, 3, 4, 5],  # 尝试不同的聚类数量
    'init': [None, 'k-means++', 'random'],  # 不同的初始化方法
    'max_iter': [None, 1,5,10,25,50, 100, 150, 200, 250],  # 不同的最大迭代次数
    'random_state': [None, 0,1,2], 
}

# 创建GridSearchCV对象
grid_search = GridSearchCV(estimator=KMeans(), param_grid=param_grid, cv=5)

# 执行网格搜索
grid_search.fit(data)

# 输出最佳参数
print("Best Parameters:", grid_search.best_params_)

# 获取最佳参数的KMeans模型
best_kmeans_model = grid_search.best_estimator_

# 使用最佳模型进行聚类
best_kmeans_model.fit(data)
best_labels = best_kmeans_model.labels_

# 计算评估指标
silhouette = silhouette_score(data, best_labels)
ch_index = calinski_harabasz_score(data, best_labels)

print("Clustering Algorithm: K-Means (Tuned)")
print("Clustering Labels:", best_labels)
print("Silhouette coefficient:", silhouette)
print("Calinski-Harabasz index:", ch_index)


Best Parameters: {'init': 'k-means++', 'max_iter': 250, 'n_clusters': 5, 'random_state': None}
Clustering Algorithm: K-Means (Tuned)
Clustering Labels: [2 0 0 4 2 0 0 0 4 0 4 0 4 2 3 3 4 2 0 4 0 0 1 1 0 0 3 0 0 3 0 1 0 0 3 0 0
 0 0 0 0 0 3 0 0 2 0 0 0 2 2 0 3 0 0 3 2 3 4 4 3 3 0 0 2 0 4 0 4 4 0 0 0 0
 0 3 0 1 4 0 2 1 3 1 0 0 1 0 0 0 0 3 1 1 0 2 0 3 0 1 3 4 4 1 4 3 0 0 2 2 3
 0 0 0 1 4 4 0 1 0 0 0 4 0 0 2 2 0 0 4 0 4 0 0 4 3 0 0 4 3 0 0 0 1 4 1 0 1
 4 4 4 0 0 0 0 0 4 0 1 3]
Silhouette coefficient: 0.5772323973789996
Calinski-Harabasz index: 62.71049428576463


In [33]:
# import json

# # 0.693
# # 创建一个包含数组的字典
# data = {"best_labels": best_labels.tolist()}

# # 保存为JSON文件
# with open("best_labels.json", "w") as json_file:
#     json.dump(data, json_file)

# # 要在其他Python脚本中读取这个JSON文件，你可以使用json.load()函数。

In [34]:
# data